In [1]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model, load_model
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = r"Dataset\Train"
validation_dir = r"Dataset\Validation"
test_dir = r"Dataset\Test"

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
batch_size = 32
image_size = (256, 256)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='input')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='input')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='input')

Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


In [13]:
def autoencoder_model(image_size):
    input_img = Input(shape=(image_size[0], image_size[1], 3))
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])
    return autoencoder

In [14]:
autoencoder = autoencoder_model(image_size)
autoencoder.fit(train_generator, epochs=5, validation_data=validation_generator)

Epoch 1/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 4855s 1s/step - Precision: 0.9999 - Recall: 0.3512 - accuracy: 0.8808 - loss: 0.5112 - val_Precision: 1.0000 - val_Recall: 0.3711 - val_accuracy: 0.9054 - val_loss: 0.4974
Epoch 2/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 4702s 1s/step - Precision: 1.0000 - Recall: 0.3496 - accuracy: 0.9166 - loss: 0.4984 - val_Precision: 1.0000 - val_Recall: 0.3888 - val_accuracy: 0.9320 - val_loss: 0.4968
Epoch 3/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 5044s 1s/step - Precision: 1.0000 - Recall: 0.3506 - accuracy: 0.9206 - loss: 0.4977 - val_Precision: 1.0000 - val_Recall: 0.3712 - val_accuracy: 0.9213 - val_loss: 0.4964
Epoch 4/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 4498s 1s/step - Precision: 1.0000 - Recall: 0.3501 - accuracy: 0.9261 - loss: 0.4975 - val_Precision: 1.0000 - val_Recall: 0.3631 - val_accuracy: 0.8762 - val_loss: 0.4962
Epoch 5/5
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 4259s 973ms/step - Precision: 1.0000 - Recall: 0.3508 - accuracy: 0.9288 - loss: 0.4975 - val_Precision: 1.

In [15]:
autoencoder.evaluate(test_generator)

d:\Python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


341/341 ━━━━━━━━━━━━━━━━━━━━ 95s 279ms/step - Precision: 1.0000 - Recall: 0.4359 - accuracy: 0.9191 - loss: 0.5091


[0.5090180039405823, 0.9999960064888, 0.43355584144592285, 0.9185023903846741]

In [16]:
autoencoder.save('deepfake_autoencoder.keras')

In [2]:
model = load_model('deepfake_autoencoder.keras')

In [14]:
def preprocess_image(image_bytes):
    nparr = np.frombuffer(image_bytes.read(), np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (256, 256))
    image = image / 255.0
    return image

def predict_deepfake(image, model):
    image = np.expand_dims(image, axis=0)
    predictions = model.predict(image)
    return predictions